In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report, f1_score

In [2]:
data = pd.read_csv("Raisin_Dataset.csv")
y = data.pop("Class")

# Split the data into train and validation, stratifying on the target feature.
X_train, X_val, y_train, y_val = train_test_split(data, y, stratify=y, random_state=23)

In [3]:
# Get a high level overview of the data. This will be useful for slicing.
X_train.describe()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter
count,675.000000,675.000000,675.000000,675.000000,675.000000,675.000000,675.000000
mean,87210.494815,427.650555,254.414345,0.779895,90407.262222,0.701092,1159.625772
std,38388.571707,110.506268,49.752074,0.088938,39602.352484,0.050807,261.820857
min,25387.000000,225.629541,144.618672,0.348730,26139.000000,0.454189,619.074000
25%,59032.500000,343.732369,218.692197,0.740516,61466.500000,0.671134,964.835500
50%,79057.000000,405.936594,247.352044,0.797864,81779.000000,0.709949,1117.107000
75%,103790.500000,493.185891,280.180509,0.840452,108022.500000,0.735886,1302.416500
max,235047.000000,843.956653,492.275279,0.923770,239093.000000,0.830632,2253.557000


In [4]:
lr = LogisticRegression(max_iter=1000, random_state=23)
lb = LabelBinarizer()

# Binarize the target feature.
y_train = lb.fit_transform(y_train)
y_val = lb.transform(y_val)

# Train Logistic Regression.
lr.fit(X_train, y_train.ravel())

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=23, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [5]:
# Use sklearn's classification report to get an overall view of our classifier.
print(classification_report(y_val, lr.predict(X_val)))

             precision    recall  f1-score   support

          0       0.88      0.84      0.86       112
          1       0.85      0.88      0.87       113

avg / total       0.86      0.86      0.86       225



# Slices

In [8]:
from sklearn.metrics import f1_score

slices = [
    ("MajorAxisLength", 427.7),
    ("MinorAxisLength", 254.4),
    ("ConvexArea", 90407.3),
]

for feature, threshold in slices:
    print(f"\nF1 score on {feature} slices:")

    row_slice = X_val[feature] >= threshold
    print(f1_score(y_val[row_slice], lr.predict(X_val[row_slice])))

    row_slice = X_val[feature] < threshold
    print(f1_score(y_val[row_slice], lr.predict(X_val[row_slice])))


F1 score on MajorAxisLength slices:
0.235294117647
0.915887850467

F1 score on MinorAxisLength slices:
0.722222222222
0.892307692308

F1 score on ConvexArea slices:
0.333333333333
0.89497716895


From the above slices, we see that our model seems to perform consistently better on raisins that are smaller than average versus ones that are larger than average.

Looking at the summary statistics, we see that for nearly every measure the median is smaller than the mean. So more than 50% of our raisins are below the average which is also where our model is strongest. 
This indicates that we might want more data on larger raisins.

# Model Card

### Model Details

Logistic Regresion model using default scikit-learn hyperparameters. Trained with sklearn version 0.24.1.

### Intended Use

For classifying two types of raisins from Turkey.

### Metrics

F1 classification with a macro average of 0.85, 0.84 for the minority class, and 0.85 for the majority class.

When analyzing across data slices, model performance is higher for raisins below the average size and much lower for raisins above the average.

### Data

Raisin dataset acquired from the UCI Machine Learning Repository: https://archive.ics.uci.edu/ml/datasets/Raisin+Dataset

Originally from: Cinar I., Koklu M. and Tasdemir S., Classification of Raisin Grains Using Machine Vision and Artificial Intelligence Methods. Gazi Journal of Engineering Sciences, vol. 6, no. 3, pp. 200-209, December, 2020.

### Bias

The majority of raisins are below the average size. This could be a potential source of bias but more subject matter expertise may be necessary. Note to students: this is a useful call out, and in a real-world scenario should prompt you to engage in collaboration with subject matter experts so you can flesh this out.